**Mohammed errakho - adil oussidi**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from PIL import Image
import glob

import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from torchsummary import summary

In [ ]:
root = '/kaggle/input/automatic-classification-of-fetal-ultrasound-brain'
files ='/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/data/data/'
train_data = '/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/train.csv'

In [ ]:
sample = pd.read_csv('/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/sample_submission.csv') 
test = pd.read_csv('/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/test.csv')

In [ ]:
images_files = glob.glob(files + '/*')
data = pd.read_csv(train_data)

In [ ]:
def Idtofile(ID):
    for i in images_files:
        if ID in i :
            return i


In [ ]:
data['file_path'] = data['image_name'].map(Idtofile)

In [ ]:
def func():
    d={},c=0
    for i in data['class'].unique():
        d[i]=c
        c+=1
    return True

In [ ]:
func()

In [ ]:
datas = pd.get_dummies(data,columns=['class'])
datas[['class_Other','class_Trans-cerebellum','class_Trans-thalamic','class_Trans-ventricular']].values[5]

In [ ]:
class Ultrasound(Dataset):
    def __init__(self, data):
        self.fpaths = data['file_path'].values
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        self.targets = data[['class_Other','class_Trans-cerebellum','class_Trans-thalamic','class_Trans-ventricular']].values
    def __len__(self): return len(self.fpaths)
    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = (cv2.imread(f)[:,:,::-1])
        im = cv2.resize(im, (224,224))
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.normalize(im) 
        return im.float().to(device), torch.tensor([target]).float().to(device)

In [ ]:
datase = Ultrasound(datas)
device

In [ ]:
im, label = datase[1124]
plt.imshow(im.permute(1,2,0).cpu())
print(label)

In [ ]:
def get_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.fc = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 4),
    nn.Softmax())
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer

In [ ]:
model, criterion, optimizer = get_model()
summary(model, torch.zeros(1,3,224,224))

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    y=y.squeeze(1)
    batch_loss = loss_fn(prediction, torch.argmax(y,1))
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    y=y.squeeze(1)
    is_correct = torch.argmax(prediction,1) == torch.argmax(y,1)
#     print(is_correct)
    return is_correct.cpu().numpy().tolist()

In [ ]:
train = np.random.choice(datas['file_path'].values,2450,replace=False)
train_data = datas.loc[datas['file_path'].isin(train)]
test_data = datas.loc[~datas['file_path'].isin(train)]

In [ ]:
def get_data():
    train = Ultrasound(train_data)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last = True)
    val = Ultrasound(test_data)
    val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last = True)
    return trn_dl, val_dl

In [ ]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
data['class'].value_counts(normalize=True)

In [ ]:
test['file_path'] = test['image_name'].map(Idtofile)
test

In [ ]:
new_d={i:j for (j,i) in d.items()}
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

In [ ]:
test['class'] = 'Trans-thalamic'
test = test.drop('file_path',1)

In [ ]:
test.to_csv('submission.csv',index=False)